In [1]:
!pip install bitsandbytes==0.43.0
!pip install -qqq -U git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, BitsAndBytesConfig
import transformers
import torch
import bitsandbytes as bnb
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)

In [3]:
config = PeftConfig.from_pretrained("wongsodillon/feedback-generator-falcon7b")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, "wongsodillon/feedback-generator-falcon7b")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
generation_config = model.generation_config
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [9]:
device = "cuda:0"

sample_prompt = """Imagine you are an interviewer interviewing a data science candidate and you are trying to give feedback based on the candidate's answer on the technical interview. Make sure to provide constructive feedback.
The question you asked: What is association rule learning?
The candidate's Answer: Association rule learning finds relationships between items in a dataset by generating if-then rules. However, it often generates redundant rules, making it difficult to interpret the most important associations.
Your feedback: """

encoding = tokenizer(sample_prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids=encoding.input_ids,
      attention_mask=encoding.attention_mask,
      generation_config=generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Imagine you are an interviewer interviewing a data science candidate and you are trying to give feedback based on the candidate's answer on the technical interview. Make sure to provide constructive feedback.
The question you asked: What is association rule learning?
The candidate's Answer: Association rule learning finds relationships between items in a dataset by generating if-then rules. However, it often generates redundant rules, making it difficult to interpret the most important associations.
Your feedback:  The candidate's explanation of association rule learning is incomplete, as it fails to mention the common application in recommendation systems, where it helps identify user preferences by analyzing the items they interact with. To better explain this application, the candidate should mention the importance of identifying user preferences in recommendation systems.
